In [2]:
import numpy as np
import cvxpy as cp
from cvxpy import*
import time

def gen_ratings(n,m,p):
    """
    Inputs:
        n: the number of users
        m: the number of movies
        p: the number of observed ratings
    Returns:
        Y: a list of p ratings in {-1, 1}
        S: a list of p tuples (i,j), 0 <= i < n, 0 <= j < m

        For 0 <= k < p, y[k] is the rating given by user S[k][0]
        for movie S[k][1]
    """
    if p > m*n:
        print('You asked for more ratings than m*n!')
        return [],[]
    np.random.seed(0) # makes this deterministic
    Y = 2.*np.random.randint(0,2,(n,m)) - 1. # generates +1/-1 ratings
    mask = np.random.rand(n,m) # samples an nxm array of random numbers in [0,1]
    cutoff = np.sort(mask.reshape(-1))[-p] # finds the pth largest number in mask
    # mask = 1*(mask <= cutoff) # mask[i,j] = 1 for p of the i,j, 0 everywhere else
    Y = Y[mask >= cutoff] # ratings to be returned
    S = np.argwhere(mask >= cutoff) # indices of ratings
    return Y, S



In [13]:
def solving_SDP(n,m,p,Y,S):
    start_time = time.time()


    A = cp.Variable((n,n), PSD = True)  
    B = cp.Variable((m,m), PSD = True)  
    X = cp.Variable((n,m))  
    V = cp.bmat([[A, X], [X.T, B]])
    objective =  cp.maximum(cp.max(cp.diag(A)),cp.max(cp.diag(B)))

    constraints = [V >> 0]
    constraints += [
    X[S[i][0],S[i][1]] * Y[i] >= 1 for i in range(p)
]
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve()
    print("Optimal value of the rate matrix is \n", X.value)



    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f'elapsed time is \n {elapsed_time}')


    

In [19]:
n=100
m=100
p=10000

Y,S = gen_ratings(n,m,p)

solving_SDP(n,m,p,Y,S)

Optimal value of the rate matrix is 
 [[-1.12463858  0.9999901   1.00000185 ...  1.64752088  1.00000388
  -0.99997938]
 [ 1.00002011 -1.61992934 -0.99999409 ...  0.99998326 -1.00000943
  -0.99999453]
 [ 1.0000321  -0.99998749  1.00000304 ... -1.00002174  0.99999863
   1.00001208]
 ...
 [ 0.99998531 -0.99999481  0.99999354 ...  1.00000841  1.00000066
   0.99998   ]
 [ 0.99996721  0.99999461  0.99999281 ...  1.00002579 -1.00000099
  -1.00002207]
 [ 0.99999574  0.9999978   0.99999305 ... -1.00000481  0.99999495
  -1.04863923]]
elapsed time is 
 19.006376266479492
